In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

from pathlib import Path
import os
import sys
from datetime import datetime, date


In [15]:
result = Path('result/')
report = Path('report/')
instances = Path('../../data/csifa')

In [16]:
def read_instance(file_name):
	
	arq = open(file_name)
	
	N = int(arq.readline())
	
	PR  = [0]*N
	PP  = [0]*N

	FR = [float(arq.readline())]*N
	FP = [float(arq.readline())]*N

	HR = [float(arq.readline())]*N
	HP = [float(arq.readline())]*N

	D = [int(i) for i in arq.readline().split()]
	
	R = [int(i) for i in arq.readline().split()]

	C = float(arq.readline().rstrip('\n'))
	
	return N, PP, PR, FP, FR, HR, HP, D, R, C

In [ ]:
MAX_CPU_TIME = 60.0
EPSILON = 1e-6

def clsr_mc_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C):

	CP = np.zeros(N)
	CR = np.zeros(N)
	#rrl = np.zeros(N)
	#dl = np.zeros(N)
	#sdl = (np.zeros((N,N))).tolist()
	KP = 0
	KR = 0 

	for i in range(N):
		CP[i] = PP[i]
		for j in range(i,N):
			CP[i] = CP[i] + HP[j]

#	for l in range(0,N):
#		if l < 1:
#			rrl[l] = R[l]
#		else:
#			rrl[l] = R[l] + max(0.0, rrl[l-1]-D[l-1])
#
#		dl[l] = max(0.0,D[l]-rrl[l])
#
#	for k in range(0,N):
#		sdl[k][k] = dl[k]
#		for j in range(k+1,N):
#			sdl[k][j] = sdl[k][j-1] + dl[j]

	for i in range(N):
		CR[i] = PR[i]
		for j in range(i,N):
			CR[i] += HP[j]
		for j in range(i,N):
			CR[i] -= HR[j]

	for i in range(N):
		aux = 0 
		for j in range(i+1):
			aux += D[j]
		KP += HP[i]*aux

	for i in range(N):
		aux = 0
		for j in range(i+1):
			aux += R[j]
		KR += HR[i]*aux
	K = KR - KP

	try:

		# create model
		model = gp.Model("clsr_mc_mip")

		# create variables		
		wp = model.addVars([(i,j) for i in range(N) for j in range(i,N)], vtype=GRB.CONTINUOUS, name="wp")
		wr = model.addVars([(i,j) for i in range(N) for j in range(i,N)], vtype=GRB.CONTINUOUS, name="wr")
		vor = model.addVars([(i,j) for i in range(N) for j in range(i,N)], vtype=GRB.CONTINUOUS, name="vor")
		xp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xp")
		xr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xr")
		yp = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yp")
		yr = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yr")

		# set objective
		#obj = gp.quicksum(xp[i]*CP[i] for i in range(N))
		#obj += gp.quicksum(yp[i]*FP[i] for i in range(N))
		#obj += gp.quicksum(xr[i]*CR[i] for i in range(N))
		#obj += gp.quicksum(yr[i]*FR[i] for i in range(N)) 
		#obj += K
		
		obj = 0.0
		for i in range(N):
			obj += xp[i]*CP[i]
			obj += yp[i]*FP[i]
			obj += xr[i]*CR[i]
			obj += yr[i]*FR[i]
		obj += K
		
		model.setObjective(obj, sense = GRB.MINIMIZE)

		for i in range(N): # 37
			ctr = 0.0
			for j in range(i+1):
				ctr += wp[j,i]
				ctr += wr[j,i]
			model.addConstr(ctr >= D[i])

		for i in range(N): # 38
			ctr = 0.0
			for j in range(i+1):
				ctr+= vor[j,i]
			for j in range(i,N):
				ctr+=(-wr[i,j])
			model.addConstr(ctr == 0)

		for i in range(N): #39
			ctr =0.0
			for j in range(i,N):
				ctr += vor[i,j]
			model.addConstr(ctr <= R[i])

		for i in range(N): # 40
			for j in range(i,N):
				model.addConstr(wp[i,j] + yp[i]*(-D[j]) <= 0)

		for i in range(N): # 41
			for j in range(i,N):
				model.addConstr(wr[i,j] + yr[i]*(-min(SR[0][i],D[j])) <= 0)

		for i in range(N): # 42
			for j in range(i,N):
				model.addConstr(vor[i,j] + yr[j]*(-R[i]) <= 0)

		for i in range(N):
			ctr = xp[i]
			for j in range(i,N):
				ctr += (-wp[i,j])
			model.addConstr(ctr == 0)

		for i in range(N): # 44
			#ctr = 0.0
			ctr = xr[i]
			for j in range(i,N):
				ctr += (-wr[i,j])
			model.addConstr(ctr == 0)

		#model.addConstrs(xp[i] <= yp[i]*SD[i][N-1] for i in range(N))

		#model.addConstrs(xr[i] <= yr[i]*min(SR[0][i],SD[i][N-1]) for i in range(N))

		model.addConstrs(xp[i] <= yp[i]*min(SD[i][N-1],C) for i in range(N))

		model.addConstrs(xr[i] <= yr[i]*min(SR[0][i],SD[i][N-1],C) for i in range(N))

		model.addConstrs(xp[i] + xr[i] <= C for i in range(N))

		#if lsdbar == 1:
		#	model.addConstrs(
		# 		(gp.quicksum(xp[l] for l in range(i)) 
		# 		+ gp.quicksum(yp[l]*sdl[l][j] for l in range(i,j+1))) >= 
		# 		sdl[0][j] for i in range(N) for j in range(i,N))

		# export .lp
		model.write("clsrp_mc_model_c52_1.mps")

		# parameters 
		model.setParam(GRB.Param.TimeLimit, MAX_CPU_TIME)
		model.setParam(GRB.Param.MIPGap, EPSILON)
		model.setParam(GRB.Param.Threads, 1)
		#model.setParam(GRB.Param.Cuts, -1)
		#model.setParam(GRB.Param.Presolve, -1)

		# relax model
		#for v in model.getVars():
		#	v.setAttr('vtype', 'C')

		model.setParam('OutputFlag', 0)
		#model.setParam('Heuristics', 0)
		model.setParam("LogToConsole", 0)

		# optimize model
		model.optimize()

		# get status
		tmp = 0
		if model.status == GRB.OPTIMAL:
			tmp = 1

		# get value
		objval = model.ObjVal
		objbound = model.ObjBound
		mipgap = model.MIPGap
		runtime = model.Runtime
		nodecount = model.NodeCount

	except gp.GurobiError as e:
		print('Error code ' + str(e.errno) + ': ' + str(e))

	return objval, objbound, mipgap, runtime, nodecount, tmp


In [ ]:
def main(file_name):
	
	N, PP, PR, FP, FR, HR, HP, D, R, C = read_instance(os.path.join(instances,file_name))

	SD = (np.zeros((N,N))).tolist()
	SR = (np.zeros((N,N))).tolist()

	for  i in range(N):
		SD[i][i] = D[i]
		SR[i][i] = R[i]
		for j in range(i+1,N):
			SD[i][j] = SD[i][j-1] + D[j]
			SR[i][j] = SR[i][j-1] + R[j]

	bestsol, bestbound, gap, runtime, numnode, tmp = clsr_mc_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C)
	
	arquivo = open(os.path.join(result,'clsr_mc_mip.csv'),'a')
	
	arquivo.write(file_name+';'
			   +str(round(bestsol,2))+';'
			   +str(round(bestbound,2))+';'
			   +str(round(gap,2))+';'
			   +str(round(runtime,2))+';'
			   +str(round(numnode,2))+';'
			   +str(round(tmp,2))+'\n')
	arquivo.close()

In [34]:
if __name__== "__main__" :

	for dim in [52]:
		for id in range(1,2):
			datafile = f"c{dim}_{id}.txt"
			print(f"Resolvendo instancia {datafile}")			
			main(datafile)

Resolvendo instancia c52_1.txt
Set parameter LogFile to value "gurobi_log.log"
Set parameter LogToConsole to value 0


## Table of results

In [9]:
data=f'result/clsr_std_mip.csv'

df = pd.DataFrame()
df = pd.read_csv(data,header=None,sep=';')

tab = pd.DataFrame()
tab = pd.concat([tab, df], ignore_index=True)
tab.columns = ['instance','objval','objbound','mipgap','time','nodes','opt']

resume = pd.DataFrame({
    'instance':f"resume",
    'objval':tab["objval"].mean(),
    'objbound':tab["objbound"].mean(),
    'mipgap':tab['mipgap'].mean(),
    'time':tab['time'].mean(),
    'nodes':tab['nodes'].mean(),
    'opt':tab['opt'].sum(),
     },index=[f"uls_mip"]
)

tab = pd.concat([tab, resume], ignore_index=True)

tab["objval"] = tab["objval"].round(2)
tab["objbound"] = tab["objbound"].round(2)
tab["mipgap"] = tab["mipgap"].round(2)
tab["time"] = tab["time"].round(2)
tab["nodes"] = tab["nodes"].round(2)
tab["opt"] = tab["opt"].round().astype('Int64')



In [10]:
tab

,instance,objval,objbound,mipgap,time,nodes,opt
0,c52_1.txt,9768.80,9626.26,0.01,60.0,41598.0,0
1,c52_2.txt,10144.60,9916.57,0.02,60.0,53619.0,0
2,c52_3.txt,9347.80,9297.25,0.01,60.0,60169.0,0
3,c52_4.txt,10355.40,10227.52,0.01,60.0,62989.0,0
4,c52_5.txt,11111.50,11062.62,0.00,60.0,77239.0,0
5,resume,10145.62,10026.04,0.01,60.0,59122.8,0


In [11]:
#tab
print(
    tab[['instance','objval','objbound','mipgap','time','nodes','opt']].
    to_latex(index=False,float_format="%.2f")
)

\begin{tabular}{lrrrrrr}
\toprule
instance & objval & objbound & mipgap & time & nodes & opt \\
\midrule
c52_1.txt & 9768.80 & 9626.26 & 0.01 & 60.00 & 41598.00 & 0 \\
c52_2.txt & 10144.60 & 9916.57 & 0.02 & 60.00 & 53619.00 & 0 \\
c52_3.txt & 9347.80 & 9297.25 & 0.01 & 60.00 & 60169.00 & 0 \\
c52_4.txt & 10355.40 & 10227.52 & 0.01 & 60.00 & 62989.00 & 0 \\
c52_5.txt & 11111.50 & 11062.62 & 0.00 & 60.00 & 77239.00 & 0 \\
resume & 10145.62 & 10026.04 & 0.01 & 60.00 & 59122.80 & 0 \\
\bottomrule
\end{tabular}

